In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import backward as bk
import forward as fw
import math

In [2]:
def backward_uniform_rand(N, theta, u, d, p, default_value, eps_left, eps_right):
    pi = ((((p*(u-1))/((1-p)*(1-d)))**theta)-1)/((u - 1)+(((p*(u-1))/((1-p)*(1-d)))**theta)*(1 - d))
    up_times = bk.up_times(N)
    down_times = bk.down_times(N)
    prm_theta = bk.prm_theta(N, theta)
    
    up_r = bk.up_r(N, u)
    up_r_uniform_rand = list(map(lambda x,y: list(map(lambda a,b: a+b, x,y)), up_r, bk.uniform_rand(N, seed, eps_left, eps_right)))

    
    down_r = bk.down_r(N, d)
    down_r_uniform_rand = list(map(lambda x,y: list(map(lambda a,b: a+b, x,y)), down_r, bk.uniform_rand(N, seed, eps_left, eps_right)))
    
    prob = bk.prob(N, p)
    prm_M = bk.prm_M(N, pi, prm_theta, up_r_uniform_rand, down_r_uniform_rand, prob)
    value = bk.value(N, pi, default_value, up_r_uniform_rand, down_r_uniform_rand)
    wealth = bk.wealth(N, prm_theta, prm_M, value)
    utility = bk.utility(N, prm_theta, value)
    exp_utility = bk.exp_utility(N, up_times, down_times, prob, utility)
    Wn = bk.Wn(N, up_times, down_times, prob, wealth)
    
    df_backward = pd.DataFrame([up_times, down_times, prm_theta, up_r_uniform_rand, down_r_uniform_rand, prob, prm_M, value, wealth, utility, exp_utility, Wn]).T
    df_backward.columns = ['up_times', 'down_times', 'prm_theta', 'up_r', 'down_r', 'prob', 'prm_M', 'value', 'wealth', 'utility', 'exp_utility', 'Wn']
    
    return df_backward#df_backward['exp_utility'][N], df_backward['wealth'][0] 

In [3]:
def forward_uniform_rand(N, theta, u, d, p, default_value, eps_left, eps_right):
    up_times = fw.up_times(N)
    down_times = fw.down_times(N)
    prm_theta = fw.prm_theta(N, theta)
    
    up_r = fw.up_r(N, u)
    up_r_uniform_rand = list(map(lambda x,y: list(map(lambda a,b: a+b, x,y)), up_r, fw.uniform_rand(N, seed, eps_left, eps_right)))
    
    down_r = fw.down_r(N, d)
    down_r_uniform_rand = list(map(lambda x,y: list(map(lambda a,b: a+b, x,y)), down_r, fw.uniform_rand(N, seed, eps_left, eps_right)))
    
    prob = fw.prob(N, p)
    prm_q = fw.prm_q(N, up_r_uniform_rand, down_r_uniform_rand)
    prm_a = fw.prm_a(N, prob, prm_q)
    prm_b = fw.prm_b(N, prm_q)
    prm_c = fw.prm_c(N, prob, prm_q)
    delta = fw.delta(N, prm_theta, prm_a, prm_b, prm_c)
    pi = fw.pi(N, prm_theta, up_r_uniform_rand, prob, prm_q, delta)
    value = fw.value(N, default_value, up_r_uniform_rand, down_r_uniform_rand, pi)
    U0 = fw.U0(N, prm_theta, value)
    I0 = fw.I0(N, prm_theta, value)
    I1 = fw.I1(N, delta, I0)
    U1 = fw.U1(N, prm_theta, delta, U0)
    exp_U1 = fw.exp_U1(N, up_times, down_times, prob, U1)
    
    df_forward = pd.DataFrame([up_times, down_times, prm_theta, up_r_uniform_rand, down_r_uniform_rand, prob, prm_q, prm_a, prm_b, prm_c, delta, pi, value, U0, I0, I1, U1, exp_U1]).T
    df_forward.columns = ['up_times', 'down_times', 'prm_theta', 'up_r', 'down_r', 'prob', 'prm_q', 'prm_a', 'prm_b', 'prm_c', 'delta', 'pi', 'value', 'U0', 'I0', 'I1', 'U1', 'exp_U1']
    
    return df_forward#df_forward["exp_U1"][N]

In [4]:
seed = 12345
N = 10
theta = 1.2
u = 1.2
d = 0.8
p = 0.4
default_value = 500
eps_left = -0.01
eps_right = 0.01

In [9]:
backward_uniform_rand(N, theta, u, d, p, default_value, eps_left, eps_right)["up_r"][3][0]-1.2

0.008474763104008298

In [9]:
forward_uniform_rand(N, theta, u, d, p, default_value, eps_left, eps_right)["up_r"][3]

[1.2084747631040083,
 1.1974784235458305,
 1.1930994671147948,
 1.2078468745329776,
 1.1905357945522355,
 1.1958300477732362,
 1.197974880720053,
 1.20614577416004]

In [10]:
backward_uniform_rand(N, theta, u, d, p, default_value, eps_left, eps_right)["down_r"][3][0]-0.8

0.00847476310400841